This is a demonstration of some functional programming tools for my buddy En Ess. Enjoy! I am taking some inspiration from Bird's [*Thinking Functionally with Haskell*](https://github.com/clojurians-org/haskell-ebook/blob/master/Thinking.Functionally.with.Haskell.pdf)

I bought the book, myself, but there is a pdf.

Let's begin with some basic operations. I was going to step through Bird's sudoku solver, but that is quite time-consuming. I spent a few lectures on it when teaching Haskell at Syracuse University in 2018. So let's just demonstrate "mapping over" lists and list constructions and also play with some infinite lists. Mathematicians actually think that it is fun to play with infinity.

After that, we'll take a functional programming approach to that canonical data science interview problem of generating a frequency distribution for words in a text file.

In [28]:
# used just a few of these, but do take a look at the rest
from functools import partial, reduce
from itertools import count, cycle, repeat, chain, product, permutations
from sympy import isprime # sheer convenience; I know you know it!
import string

In [6]:
# mapping over a list
this = range(100)
that = list(map(lambda x: 2*x, this))
print(that) # its just each int from the range, doubled

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198]


Note that in fact we mapped over an *iterable*.

**The `range` function returns an iterable**

In [7]:
# grabbing items from an iterable
this = range(100)
this = iter(this)
next(this), next(this)

(0, 1)

In [8]:
# doing that same thing but incorrectly - note the error
this = range(100)
next(this)

TypeError: ignored

Words matter a whole lot at *iterable* does not mean the same thing as *iterator*. We pass an iterable to `iter` and get an iterator back.

Another way to get that same list would be to filter a bigger range:

In [9]:
this = range(200)
that = list(filter(lambda x: x%2==0, this))
print(that)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198]


What if we wanted multiples of $6$? We can filter a filter, converting to a list at the end:

In [10]:
this = range(200)
that = filter(lambda x: x%2==0, this)
other = list(filter(lambda x: x%3==0, that))
print(other)

[0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198]


What if we wanted primes? What is cool is that we can create an 'infinite list' of primes by building from the `count` function that we imported - it will generate all of the natural numbers for us!

In [11]:
nats = count(1)
next(nats), next(nats), next(nats)

(1, 2, 3)

You might guess that we have to reset `nats` and you would be right.

In [12]:
# resetting our generator
nats = count(1)

# our infinite 'list' of primes
primes = filter(isprime, nats)
type(primes) # what even is it?

filter

It cannot really be that easy, can it? Well, we cheated quite a lot by importing stuff. So yes, it is that easy:

In [13]:
# the first three primes
next(primes), next(primes), next(primes)

(2, 3, 5)

(Say what would happen if you reran the above cell and then rerun it.)

Finally, let's do that text parsing task we talked about, where we return a frequency distribution on words, but in the functional style! Is this implementation better than a more conventional one? Nah. It's just an exercise and I hope it will stir up some questions. For example, it would be useful to spend some time analyzing the time complexity of the function.

We will use `reduce` and `chain` here - look those up if it is not obvious how they are working, or just to learn more. There is always more.

First, let's handle the pesky task of creating a text file. We won't worry about the functional paradigm here because we just need some dummy text to work with.

In [14]:
!pip install lorem # installing lorem

We import `lorem`.

In [15]:
import lorem

We get some text from it.

In [16]:
t = lorem.text()

In [17]:
print(t)

Etincidunt numquam adipisci magnam amet sit velit. Magnam ipsum aliquam magnam tempora sed magnam etincidunt. Est labore aliquam neque porro. Amet quaerat adipisci dolore adipisci sit velit. Tempora tempora ipsum non labore magnam. Quaerat sed quaerat etincidunt sit. Neque porro dolore non non neque dolor.

Ipsum labore adipisci est tempora magnam. Labore magnam magnam non porro. Ipsum quaerat neque amet. Ut consectetur voluptatem sed dolor. Quiquia magnam magnam dolore tempora. Amet ipsum sed eius eius. Consectetur neque velit consectetur quaerat sit quiquia non.

Dolorem velit labore dolore. Non tempora neque quiquia amet dolore magnam quiquia. Consectetur neque aliquam porro. Amet neque numquam adipisci neque quaerat aliquam neque. Amet etincidunt modi dolorem magnam tempora aliquam numquam. Etincidunt consectetur dolore amet dolore.

Tempora non est labore voluptatem. Ipsum porro etincidunt etincidunt. Etincidunt amet ut velit ut amet eius magnam. Quiquia tempora sed modi etincidun

In [18]:
# if we want nonempty lines, will this do the trick?
len(list(filter(lambda x: x, t.split('\n'))))

6

(The above line is extremely dense - take your time thinking about it. In particular, what in the world is going on with the lambda expression passed to `filter`? Learn how python treats non-bools as bools)

In [19]:
# yes, that does it
lines = list(filter(lambda x: x, t.split('\n')))

In [20]:
lines

['Etincidunt numquam adipisci magnam amet sit velit. Magnam ipsum aliquam magnam tempora sed magnam etincidunt. Est labore aliquam neque porro. Amet quaerat adipisci dolore adipisci sit velit. Tempora tempora ipsum non labore magnam. Quaerat sed quaerat etincidunt sit. Neque porro dolore non non neque dolor.',
 'Ipsum labore adipisci est tempora magnam. Labore magnam magnam non porro. Ipsum quaerat neque amet. Ut consectetur voluptatem sed dolor. Quiquia magnam magnam dolore tempora. Amet ipsum sed eius eius. Consectetur neque velit consectetur quaerat sit quiquia non.',
 'Dolorem velit labore dolore. Non tempora neque quiquia amet dolore magnam quiquia. Consectetur neque aliquam porro. Amet neque numquam adipisci neque quaerat aliquam neque. Amet etincidunt modi dolorem magnam tempora aliquam numquam. Etincidunt consectetur dolore amet dolore.',
 'Tempora non est labore voluptatem. Ipsum porro etincidunt etincidunt. Etincidunt amet ut velit ut amet eius magnam. Quiquia tempora sed mod

OK, we'll write that to a text file and that text file will be our starting point, just so that we are not cheating too much. We want to demonstrate that the functional paradigm really can be used to solve a real parsing problem starting from a file on disk.

In [21]:
with open('loremText.txt', 'w') as f:
  for line in lines:
    f.write('%s\n' % line)

Look for the file `loremText.txt` in your "Files" on the left if you doing this is Colab. We will start from that file, defining a function that takes an arbitrary file object and generates lines, iterables of words (string objects), from it.

In [32]:
# starting with a line generator
def lineGenerator(fileObject):
  line = fileObject.readline()
  while line:
    # strip newlines, lowercase everything and return it
    yield map(lambda x: x.lower().strip(string.punctuation), line.rstrip().split())
    line = fileObject.readline()

Look [here](https://stackoverflow.com/questions/519633/lazy-method-for-reading-big-file-in-python) to see how you might handle empty lines (which do not worry us because we already filtered them out for simplicity).

Aside: stackoverflow is your very great friend.

Now we need a function that accumulates our dictionary. It will take two arguments: a dictionary and a word. We will then pass this function to `reduce` together with our iterable of all the words in the file.

In [35]:
def updateDictionary(tally, word):
  if word in tally:
    tally[word] += 1
  else:
    tally[word] = 1
  return tally

We are ready to build our frequency distribution. Let's wrap the entire thing in a tidy little function that takes the name of a text file as its sole input.

In [36]:
def getCounts(thatFile):
  with open(thatFile) as f:
    allText = chain.from_iterable(lineGenerator(f))
    wordCounts = reduce(updateDictionary, allText, {})
  return wordCounts

And call that function:

In [37]:
getCounts('loremText.txt')

{'adipisci': 10,
 'aliquam': 9,
 'amet': 15,
 'consectetur': 9,
 'dolor': 10,
 'dolore': 8,
 'dolorem': 8,
 'eius': 7,
 'est': 7,
 'etincidunt': 13,
 'ipsum': 11,
 'labore': 9,
 'magnam': 16,
 'modi': 8,
 'neque': 14,
 'non': 13,
 'numquam': 9,
 'porro': 9,
 'quaerat': 12,
 'quiquia': 10,
 'quisquam': 2,
 'sed': 9,
 'sit': 7,
 'tempora': 11,
 'ut': 5,
 'velit': 9,
 'voluptatem': 5}

Looks great! Tight as fuck, I am great.

If I have done this correctly, then all of this should work in a [lazy](https://en.wikipedia.org/wiki/Lazy_evaluation) fashion. In particular, it should work fine even on gigantic text files provided that none of their individual lines are too big.

Challenge: what if there were no such guarantee on the size of lines in the input file?